<a href="https://colab.research.google.com/github/learning-stack/Colab-ML-BenProjects/blob/master/AskFrancisFAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ask Francis FAQ
FLL (web@fll.cc)


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-BenProjects/blob/master/AskFrancisFAQ/AskFrancisFAQ.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-BenProjects/blob/master/AskFrancisFAQ/AskFrancisFAQ.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [16]:
import math
import os
from random import sample
import cPickle as pickle

from scipy.spatial.distance import cdist
import json
import numpy as np


!pip install indicoio
import indicoio
indicoio.config.api_key = '3e308585ac44074507c9515ddd8a0d6f'

!pip install texttable
from texttable import Texttable

from tqdm import tqdm

In [0]:
faqs = {
    'My husband is a Christian and I am a Catholic and we have been married for over thirty years. Two years ago, I was diagnosed with cancer. When struggling from my sickness, I asked my husband to go to learn Catholic bible so that he can be baptized as a Catholic once I recovered. He agreed but his friends and old classmates learnt of this and said he is a traitor from his own religion. How can I comfort my husband for being called a traitor by his friends?':'Q1',
    'Some ladies wear a veil to mass but most don’t. Some wear black and some wear white. What is the meaning of the veil?':'Q2',
    'I have been trying to quit my bad habit of patronizing massage parlors for ages as I now know it is conflicting with the sixth commandment. My sex drive pushes me so hard that I use masturbation with porn videos so to avoid the action of buying sexual satisfaction. Which action is more serious?':'Q3',
    'If we live with a pagan family, can we eat the food offered for the gods and ancestors, or help them light the incense at home for the ancestor？':'Q4',
    'Can a widower be ordained priest?':'Q5',
    'Can a catholic baptise a non-catholic who is dying and unable to express his wishes concerning baptism?':'Q6',
    'Why is it that the catholic Bible has books that the other Bibles don’t contain?':'Q7',	
    'Why do we Catholics have structured prayers other than the “Our Father”?':'Q8',	
    'Are we all affected by our ancestors’ sins if they have not paid off yet?':'Q9',	
    'Is Christianity a made-up religion by people who want to change the society and risked their lives doing this? Can it be fake? I know there are historical evidence of Jesus from historians but can they lie too? Can the apostles lie? Can the miracles and contents of the Bible be fake? I really do want to believe because I know that a lot of good things can happen to me and the service I can provide for God but I really have a lot of doubts. Please help Fr. Francis!':'Q10',
	'Why do we need to pray? Do praying really help us?':'Q11',
	'Is it true a divorced person not allowed to be baptized? How about for those who are remarried? Why?':'Q12',
	'What should we do with the palm branches we took home on Palm Sunday? Shall we bring them back to church? And when? Is it that OK to fold the palm branches into crosses?':'Q13',
	'When and why the statues and cross in the church are covered or removed during Lent?':'Q14',
	'What are the various Catholic churches with names like Maronite, Coptic and Slovak Catholic Church. In the Nicene Creed,”I believe one holy catholic and apostolic Church”, so are churches just like CMCC serving the ethnic groups? Are these churches legit for Roman Catholics? How about Eastern and Orthodox? ':'Q15',
	'I have been getting acupuncture treatments for sometime now but have been getting mixed comments from different people in regards to acupuncture not being accepted in our Catholic faith. Could you please clarify?':'Q16',
	'My siblings are planning a family vacation for our elderly parents. We are Catholic but not all practicing. They want to go to a remote resort where I may not be able to find a church to fulfill my Sunday obligation. I am torn between 1) going for the trip & missing Sunday Mass, or 2) skipping the trip & disappointing my elderly parents. Any advice?':'Q17',
	'We are Catholic and received an invitation from our cousin to attend his daughter’ s same sex marriage in Toronto. Are we supposed to refuse to attend or not?':'Q18',
	'How do we have god?':'Q19',
	'When Angel Gabriel visited Zechariah and Mary, both of them did not understand how things could happen the way Gabriel announced. Though they asked similar questions, Zechariah ended up losing his power of speech whereas for Mary, the angel just left her. Why did they have such different consequences? ':'Q20',
	'Mark 7:1-13 mentions that other than the ten commandments, the Pharisees added extra laws and rules that Jews should follow. Is our church today not doing the same thing, such as harsh rules on the divorce and mandatory mass of some feast days?':'Q21',
	'We all have accumulated a lot of religious objects, such as prayer cards, rosaries, and pictures of Saints. But some are either worn out or broken. What is the most appropriate way of disposing them ? I have heard that it is a sin to toss them into the garbage ? ':'Q22',
	'After the priest blessed the Eucharist, some people would stand up, while others would prefer to continue to kneel. Which way is correct? ':'Q23',
	'If a person passed away suddenly leaving his/her family on earth, would the departed miss the family if he/she is in heaven? If the departed knows his/her family member(s) will head towards spiritual death, would he/she be sad and concerned in heaven? If yes, is there any sadness in heaven?':'Q24',
	'Pope Francis has said: “Hope in idols are illusions.” What about religious objects and statues that adorned the Catholic churches and in lay people’s homes?':'Q25',
	'Is it true that Christians should not do yoga? Is it fine to do yoga if I don’t say their words or join their meditation? Is Yoga really an invitation to demonic possession because of its moves and postures?':'Q26',
	'Can a non catholic baptize a dying person if the dying person expresses a wish to become catholic?':'Q27',
	'I often thought Holy Spirit is always upper case H and S, until when I read NEW AMERICAN BIBLE (NAB), and it is written as “holy Spirit”. Example would be “and the holy Spirit was upon him” (Lk 2:25). Would you please explain?':'Q28',
	'Is it true that priests must pray or do penance for all those who come to him for Confession?':'Q29',
	'The hours available (at my church) does not match my schedule. May I ask to do confession online here?':'Q30'
}

In [0]:
def make_feats(data):
    """
    Send our text data through the indico API and return each text example's text vector representation
    """
    chunks = [data[x:x+100] for x in xrange(0, len(data), 100)]
    feats = []
    # just a progress bar to show us how much we have left
    for chunk in tqdm(chunks):
        feats.extend(indicoio.text_features(chunk))
    return feats

In [0]:
def calculate_distances(feats):
    # cosine distance is the most reasonable metric for comparison of these 300d vectors
    distances = cdist(feats, feats, 'cosine')
    return distances

In [0]:
def similarity_text(idx, distance_matrix, data, n_similar=5):
    """
    idx: the index of the text we're looking for similar questions to
         (data[idx] corresponds to the actual text we care about)
    distance_matrix: an m by n matrix that stores the distance between
                     document m and document n at distance_matrix[m][n]
    data: a flat list of text data
    """
    t = Texttable()
    t.set_cols_width([50, 20])

    # these are the indexes of the texts that are most similar to the text at data[idx]
    # note that this list of 10 elements contains the index of text that we're comparing things to at idx 0
    sorted_distance_idxs = np.argsort(distance_matrix[idx])[:n_similar] # EX: [252, 102, 239, ...]
    # this is the index of the text that is most similar to the query (index 0)
    most_sim_idx = sorted_distance_idxs[1]

    # header for texttable
    t.add_rows([['Text', 'Similarity']])
    print t.draw()

    # set the variable that will hold our matching FAQ
    faq_match = None

    for similar_idx in sorted_distance_idxs:
        # actual text data for display
        datum = data[similar_idx]

        # distance in cosine space from our text example to the similar text example
        distance = distance_matrix[idx][similar_idx]

        # how similar that text data is to our input example
        similarity =  1 - distance

        # add the text + the floating point similarity value to our Texttable() object for display
        t.add_rows([[datum, str(round(similarity, 2))]])
        print t.draw()

        # set a confidence threshold
        if similar_idx == most_sim_idx and similarity >= 0.80:
                faq_match = data[most_sim_idx]
        else:
            sorry = "Sorry, I'm not sure how to respond. Let me find someone who can help you."

    # print the appropriate answer to the FAQ, or bring in a human to respond
    if faq_match is not None:
            print "A: %r" % faqs[faq_match]
    else:
            print sorry    

In [0]:
def input_question(data, feats, lang):
    # input a question
    question = raw_input("What is your question? ")
    # add the user question and its vector representations to the corresponding lists, `data` and `feats`
    # insert them at index 0 so you know exactly where they are for later distance calculations
    if question is not None:
        data.insert(0, question)
    new_feats = indicoio.text_features(question)
    feats.insert(0, new_feats)
    input_lang = 'English'
    language_dict = indicoio.language(question)
    if language_dict['Chinese'] > 0.25:
        input_lang = 'Chinese'
    lang.insert(0, input_lang)
    return data, feats, lang

In [0]:
def run():
    data = faqs.keys()
    print "FAQ data received. Finding features."
    feats = make_feats(data)
    lang = []
#    with open('faq_feats.pkl', 'wb') as f:
#        pickle.dump(feats, f)    
#    with open('faq_feats.pkl', 'rb') as f:
#        feats = pickle.load(f)
    print "Features found -- success! Calculating similarities..."
    input_results = input_question(data, feats, lang)
    new_data = input_results[0]
    new_feats = input_results[1]
    language =  input_results[2]
    print(language)
    distance_matrix = calculate_distances(new_feats)
    print "Similarities found. Generating table."
    idx = 0
    similarity_text(idx, distance_matrix, new_data)
    print '\n' + '-' * 80

In [171]:
if __name__ == "__main__":
    run()

  0%|          | 0/1 [00:00<?, ?it/s]

FAQ data received. Finding features.


100%|██████████| 1/1 [00:01<00:00,  1.35s/it]


Features found -- success! Calculating similarities...
What is your question?  什么是新时代？
['Chinese']
Similarities found. Generating table.
+----------------------------------------------------+----------------------+
|                        Text                        |      Similarity      |
+====================================================+======================+
+----------------------------------------------------+----------------------+
+----------------------------------------------------+----------------------+
|                   什么是新时代？                   |         nan          |
+====================================================+======================+
+----------------------------------------------------+----------------------+
+----------------------------------------------------+----------------------+
|         Can a widower be ordained priest?          |         nan          |
+====================================================+======================+
+-----------